In [1]:
#Nisa Pinar Ruzgar 220201050
import numpy as np
import cv2
import math

            
def spatial_dist_matrix(windowSize, sigma):             # to calculate the spatial distances only once for a nxn matrix
    spatial_matrix = [[0 for x in range(windowSize)] for y in range(windowSize)]                 # n = windowSize
    
    if windowSize % 2 == 1:
        r = int((windowSize - 1) / 2)
        sp_kernelSum = 0
        
        for i in range (0, windowSize):
            for j in range (0, windowSize):
                dist = getDist(i, j, r, r)
                val = gaussian_f(dist, sigma)
                spatial_matrix[i][j] = val
                sp_kernelSum = sp_kernelSum + val
    else:
        print("Window size should be odd.")
        
    return spatial_matrix , sp_kernelSum
   
    
def gaussian_f(dist, sigma):
    return (1/ (2 * math.pi * sigma * sigma)) * math.exp(- (dist**2) / (2 * sigma**2 ) )
    
def getDist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def bilateral_f(img, img_filtered, windowSize, sp_sigma , inten_sigma):
    if windowSize % 2 == 1:
        
        h,w = img.shape[:2]                                       # to determine the range of i and j
        spatialKernel , sp_kernelSum = spatial_dist_matrix(windowSize, sp_sigma)
        radius = int((windowSize - 1) / 2)

        for i in range (0 , h - windowSize):       # i and j are for looping through the whole image
            for j in range(0, w - windowSize):
                pixVal = 0
                Wp = 0                   # Normalization term

                x  = 0
                for r in range(i, i + windowSize):      # r and c for looping through the nxn window
                    y = 0
                    for c in range(j, j + windowSize):
                        intensity = gaussian_f( int(img[r][c]) - int(img[i+radius][j+radius] ), inten_sigma)
                        g = intensity *  spatialKernel[x][y]
                        pixVal = (pixVal + (  g * img[r][c])) 
                        Wp = Wp + g

                        y = y + 1
                    x = x + 1

                normalizedPixVal =  int (round (pixVal / Wp)) 

                if (normalizedPixVal> 255):          # check if overflow occured
                    normalizedPixVal = 255
                elif (normalizedPixVal < 0):
                    normalizedPixVal = 0 

                img_filtered[i + radius][j + radius] = normalizedPixVal

    else:
        print ("Window size should be odd.")






In [2]:
img = cv2.imread("in_img.jpg",0)
img_filtered = cv2.imread("in_img.jpg",0)

bilateral_f(img, img_filtered, 7, 10, 10)
cv2.imwrite("filtered_image_own.png",img_filtered)

img2 = cv2.imread("in_img.jpg",0)
img2 = cv2.bilateralFilter(img2,7,10,10)
cv2.imwrite("filtered_image_OpenCV.png",img2)


True

In [ ]:
# If we increase spatial sigma, larger features are smoothened more.
# If we increase intensity sigma, the bilateral filter gets closer to Gaussian convolution. 

#ödevde border handling eksik
